# A Taste of Machine Learning

How do you know if a wine is good or bad? A sommelier might tell you that the chemical properties of wine affect the taste and the quality of the wine, but what if we could test this using machine learning?

**Please note**: This exercise is based on a Jupyter notebook (an interactive environment for writing and running code) and is running in Python. To get familiar with working in Jupyter notebooks, see our 2-minute [JupyterLab Tutorial](https://www.university4industry.com/player/chapter/jupyterlab-tutorial).

<div class="alert alert-block alert-info">A cell like this indicates a question you need to answer in the Answers.txt file. Please answer the question <b>before</b> continuing through the notebook. You can <b>double click on Answers.txt</b> in the Left Sidebar now to open it in a new tab. As you go through the notebook, navigate between the tabs to answer questions.
</div>

## Table of contents

1. [Introduction](#1.-Introduction)

2. [Get familiar with the data](#2.-Get-familiar-with-the-data)

3. [Explore the data](#3.-Explore-the-data)

4. [Clean & prepare the data](#4.-Clean-&-prepare-the-data)
   
5. [Train & evaluate a model](#5.-Train-&-evaluate-a-model)

## 1. Introduction

[[ go back to the top ]](#Table-of-contents)

In this challenge, we will build a predictive model that can determine the quality of a wine. We will use a real data set of 1599 red Vinho Verde wine samples from the northern Portugal (Source: [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/wine+quality)), which includes a number of variables from tests, and a variable `Quality`, which is a score from 1-10 given by experts. Our hypothesis is that chemical properties contribute to wine quality and therefore make a wine "poor", "average", or "excellent". 

To do so we will first explore our data set and create some visualizations to gather some insights about which varaibles might contribite to wine quality. We will then create a model using a part of the data set and evaluate it to determine how effective the model would be at predciting wine qulaity when given previously unseen wines. 

By the end of this tutorial, you will get a basic understanding of how to analyze a large data set and extract some insights from it. You will also learn the importance of the training data set and its quality for creating a preditive machine learning model. 

## 2. Get familiar with the data

[[ go back to the top ]](#Table-of-contents)

Before we start exploring, we need to import some libraries that will help us with our analyses and visualizations. 

*Remember to press ***Shift+Enter*** to run each code cell.*

In [ ]:
# Import data analysis libraries
import pandas as pd
import numpy as np
import random as rnd

# Import visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Linear regression and confusion matrix
import sklearn as sk
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

# Note: this cell and some of the cells below produce no visible output
# The sucessful execution of a code cell is indicated by the number in the brackets [ ] on the left

Now, let's import our data set `winequality-red.csv` and take a look at it:

In [ ]:
# Load data from file into a new object called "wine_data"
wine_data = pd.read_csv('winequality-red.csv', sep=';')

# Show first 10 rows of data set
wine_data.head(10)

Our data set includes 11 variables (the predictors of wine quality):
- <b>Fixed acidity:</b> non-volatile acids that do not evaporate readily 
- <b>Volatile acidity:</b> acetic acid in wine which, in large quantities, leads to an unpleasant vinegar taste
- <b>Citric acid:</b> acts as a preservative to increase acidity; in small quantities, adds freshness and flavor to wines
- <b>Residual sugar:</b> the amount of sugar remaining after fermentation stops; the key is to have a perfect balance between sweetness and sourness
- <b>Chlorides:</b> the amount of salt 
- <b>Free sulfur dioxide:</b> prevents microbial growth and the oxidation 
- <b>Total sulfur dioxide:</b> amount of SO2
- <b>Density:</b> sweeter wines have a higher density
- <b>pH:</b> the level of acidity on a scale of 0–14; most wines are between 3–4
- <b>Sulfates:</b> an additive that contributes to SO2 levels and acts as an antimicrobial and antioxidant
- <b>Alcohol:</b> the fun part

There is also one variable based on sensory data (the target variable/what we want to predict):
- <b>Quality</b> (score from 1 to 10 given by experts)

Source: [Red Wine Quality Prediction using Classification and Regression Model (Medium)](https://medium.com/dev-genius/red-wine-quality-prediction-using-classification-and-regression-model-f19337821b71)


## 3. Further explore the data

[[ go back to the top ]](#Table-of-contents)

**Data visualization** can be a powerful tool that allows us to see trends and patterns and relationships between variables because our brains are very good at spotting patterns on pictures.

As our target variable, or the variable we want to predict with our model, is the quality of the wines, let's start by visualizing `quality`. 

In [ ]:
# Create histogram of the target variable (Quality)
sns.countplot(x='quality', data=wine_data)
plt.show()

<div class="alert alert-block alert-info">Pause! Answer <b>Question 1</b> in the Answers.txt file. 
    
Which 2 wine qualities are most common in our data set? Do you forsee any problems that might occur becauuse of which wine qualities are most common?</div>

We can also create a heat map to visualize which variables correlate with each other.

In [ ]:
plt.subplots(figsize=(15, 10))
sns.heatmap(wine_data.corr(), annot = True, cmap = "coolwarm")
plt.show()

The correlation coefficient ranges from –1 to 1. When it is closer to 1, there is a strong positive correlation (e.g., `quality` goes up when `alcohol` goes up). When the coefficient is closer to –1, there is a strong negative correlation (e.g, you can see a small negative correlation between the `volatile acidity` and `quality`). Finally, coefficients close to zero mean that there is no correlation.

There are a lot of numbers in this heat map so let's look a little closer at two specific variables: `pH` and `fixed acidity`.

In [ ]:
#Visualize the correlation between pH and fixed acidity

#Create a new dataframe containing only pH and fixed acidity columns 
fixedAcidity_pH = wine_data[['pH', 'fixed acidity']]

#Initialize a joint-grid with the dataframe, using seaborn library
gridA = sns.JointGrid(x="fixed acidity", y="pH", data=fixedAcidity_pH, size=6)

#Draw a regression plot in the grid 
gridA = gridA.plot_joint(sns.regplot, scatter_kws={"s": 10})

#Draws a distribution plot in the same grid
gridA = gridA.plot_marginals(sns.distplot)

This scatter plot shows that, as fixed acidity levels increase, pH levels drop. Makes sense doesn’t it? A lower pH level is an indicator of high acidity.

Let's try another one: alcohol and quality. Since there are 6 discrete categories of `quality`, we can use a bar graph to visualize this.

In [ ]:
#Visualize the correlation between alcohol and quality

#Create a new dataframe containing only alcohol and quality columns 
alcohol_quality = wine_data[['alcohol', 'quality']]

fig, axs = plt.subplots(ncols=1,figsize=(10,6))
sns.barplot(x='quality', y='alcohol', data=alcohol_quality, ax=axs)
plt.title('quality vs. alcohol')

plt.tight_layout()
plt.show()

<div class="alert alert-block alert-info">Pause! Answer <b>Question 2</b> in the Answers.txt file. 
    
Describe the relationship between the variables quality and alcohol in 1 sentence. Does this surprise you? Why or why not?</div>

## 4. Clean & prepare the data

[[ go back to the top ]](#Table-of-contents)

Cleaning a data set before using it to build a machine learning model is an important step in the data anylsis pipeline as raw data sets can skew the final model and provide misleading insights. 

There are several steps that can be taken to clean a data set:
- Identifying and correcting mistakes or errors in the data
- Dealing with missing data
- Identifying variables that are most relevant to the task and removing irrelevant ones
- Converting text labels into numbers
- Combining variables 

Let's check our data set for missing values.

In [ ]:
# Show missing values for each variable
print(wine_data.isna().sum())

We can see that we have no missing values in our data set. 

Next, let's consider whether our data set is organized in a way that allows us to answer our original question: Is a wine of poor, average or, excellent quality? At the moment, our data set has the variable `quality` which is a rating of the wine quality from 1 to 10. Since we want to classify wines into 1 of 3 categories, we can aadapt our data set to reflect this:

<b>Poor</b>: all wines rated 4 or lower\
<b>Average</b>: wines rated 5 and 6\
<b>Excellent</b>: wines rated 7 or higher

We can represent these 3 categories by numbers 1 (Poor), 2 (Average), and 3 (Excellent) in a 13th variable called `review`. 

In [ ]:
# Replace `quality` by `review`
reviews = []
for i in wine_data['quality']:
    if i >= 1 and i <= 3:
        reviews.append('1')
    elif i >= 4 and i <= 7:
        reviews.append('2')
    elif i >= 8 and i <= 10:
        reviews.append('3')
wine_data['review'] = reviews

In [ ]:
#Show first 5 rows of data set with 13th 'review' variable added
wine_data.head(5)

## 5. Train & evaluate a model

[[ go back to the top ]](#Table-of-contents)

Before we train and test our model we need to split our data set into two groups: a training set (80%), and a test set (20%). The training set will be used to build our machine learning model, and the test set will be used to see how well the model performs on new, unseen data.

In [ ]:
#Define review variable as dependent variable (y) and all other variables as independent variables (x)
y = wine_data.review
x = wine_data.drop('review', axis=1)

#Split data into training and test set
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

There are a number of models that we can use. Since we have want to solve a classification problem (Is a wine poor, average, or excellent?), we can use the logistic regression algorithm. 

In [ ]:
# Fit training data to model 
lr = LogisticRegression()
lr.fit(x_train, y_train)

# Evaluate model on test data
lr_predict = lr.predict(x_test)

# Print accuracy score 
print(lr_acc_score*100)

<div class="alert alert-block alert-info">Pause! Answer <b>Question 3</b> in the Answers.txt file. 
    
What does the accuracy score tell us about our model in terms of prediciting wine quality? 

Generally, accuracy is not suffcient for telling us whether we have a good model as it can be misleading. Let's see why by using a confusion matrix to see how well our model performs for each `review` class.

A confusion matrix compares the predicted (wine review) values by the model with the actual (wine review) values. The number of correct and incorrect predictions are summarized with numbers, broken down by class. These numbers are organized into a table or matrix to compare the predicted values and actual values. If you're interested in learning more, see [here](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62).

In [ ]:
#Create heatmap confusion matrix
ax = plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt='g');

# Set labels, title, and ticks
ax.set_xlabel('Predicted review');ax.set_ylabel('True review'); 
ax.xaxis.set_ticklabels(['Poor', 'Average', 'Excellent']); ax.yaxis.set_ticklabels(['Excellent', 'Average', 'Poor']);

The confusion matrix tells us that our model is very good at identifying average wines, but terrible at identifying poor or excellent wines. In practice, this means our model is not very useful. But why did this happen? If you recall the histogram we created earlier, the majority of the wines in the data set were rated as 5 or 6 in quality. Let's see a histogram of the variable `review` which directly corresponds to what we want our model to predict.

In [ ]:
# Create histogram of the target variable (Review)
sns.countplot(x='review', data=wine_data)
plt.show()

Now we can clearly see why our model is so bad at predicting poor (`review` 1) and excellent (`review` 2) wines. There were simply not enough wines labelled as poor or excellent for the model to "learn" how to identify them. 

This tells us two very important lessons about building a machine learning model:
1. The data used to train a model must reflect the question you are trying to answer
2. Simply having a large data set does not mean you will have a good model

In our case, we had a relatively large data set, but because the majority of data points were in one class (Average), we were unable to build a good model. 

<div class="alert alert-block alert-info">Pause! Answer <b>Question 4</b> in the Answers.txt file. 
    
How can we improve our model? 

#### Congratulations! You have completed A Taste of Machine Learning! 

#### Remember to save the Answers.txt file (via File > Save File) before you close the tutorial by clicking on the "Submit" button.